In [15]:
import os
import json
import pandas as pd

baseline_path = "/home/xuyang1/mle-bench-aide-gpt-4o-2024-mini"
output_file = "/home/xuyang1/mle-bench-aide-gpt-4o-2024-mini/res_gpt_4o_aide.csv"

data_list = []

competition_score_dict = {}

for folder in os.listdir(baseline_path):
    folder_path = os.path.join(baseline_path, folder)
    if os.path.isdir(folder_path):
        grading_file_path = None
        for file in os.listdir(folder_path):
            if file.endswith("grading_report.json"):
                grading_file_path = os.path.join(folder_path, file)
                break

        with open(grading_file_path, "r") as f:
            grading_data = json.load(f)

            for report in grading_data.get("competition_reports", []):
                data_list.append({
                    "competition_id": report.get("competition_id"),
                    "score": report.get("score"),
                    "gold_threshold": report.get("gold_threshold"),
                    "silver_threshold": report.get("silver_threshold"),
                    "bronze_threshold": report.get("bronze_threshold"),
                    "median_threshold": report.get("median_threshold"),
                    "any_medal": report.get("any_medal"),
                    "gold_medal": report.get("gold_medal"),
                    "silver_medal": report.get("silver_medal"),
                    "bronze_medal": report.get("bronze_medal"),
                    "above_median": report.get("above_median"),
                    "submission_exists": report.get("submission_exists"),
                    "valid_submission": report.get("valid_submission"),
                })
                competition_score_dict[report.get("competition_id")] = (report.get("score"), report.get("is_lower_better"))
competition_df = pd.DataFrame(data_list)
# competition_df


In [16]:
# import bo-full result
# bo_full_path = "runs/2025-01-22T08-01-25-GMT_run-group_aide-bo-full/2025-01-27T12-47-40-GMT_grading_report.json"
bo_full_path = "runs/2025-01-29T01-54-52-GMT_run-group_aide-bo-full/2025-02-03T07-17-30-GMT_grading_report.json"

with open(bo_full_path, "r") as f:
    bo_full_data = json.load(f)

data_list_bo = []

competition_score_dict_bo = {}
for report in bo_full_data.get("competition_reports", []):
    data_list_bo.append({
        "competition_id": report.get("competition_id"),
        "score": report.get("score"),
        "gold_threshold": report.get("gold_threshold"),
        "silver_threshold": report.get("silver_threshold"),
        "bronze_threshold": report.get("bronze_threshold"),
        "median_threshold": report.get("median_threshold"),
        "any_medal": report.get("any_medal"),
        "gold_medal": report.get("gold_medal"),
        "silver_medal": report.get("silver_medal"),
        "bronze_medal": report.get("bronze_medal"),
        "above_median": report.get("above_median"),
        "submission_exists": report.get("submission_exists"),
        "valid_submission": report.get("valid_submission"),
    })
    competition_score_dict_bo[report.get("competition_id")] = (report.get("score"), report.get("is_lower_better"))
bo_full_df = pd.DataFrame(data_list_bo)
# bo_full_df


In [17]:
# only use the lite competition id
competition_id_lite = bo_full_df['competition_id'].unique()
competition_df = competition_df[competition_df['competition_id'].isin(competition_id_lite)]


In [18]:
grouped_df = competition_df.groupby('competition_id').agg(
    total_competitions=('competition_id', 'size'),
    valid_submissions=('valid_submission', 'sum'),
    submission_exists=('submission_exists', 'sum'),
    any_medal=('any_medal', 'sum'),
    gold_medal=('gold_medal', 'sum'),
    silver_medal=('silver_medal', 'sum'),
    bronze_medal=('bronze_medal', 'sum'),
    above_median=('above_median', 'sum')
).reset_index()
# grouped_df.to_csv("/home/xuyang1/mle-bench-aide-gpt-4o-2024-mini/res_gpt_4o_aide_each.csv", index=False)
# grouped_df

grouped_df_bo = bo_full_df.groupby('competition_id').agg(
    total_competitions=('competition_id', 'size'),
    valid_submissions=('valid_submission', 'sum'),
    submission_exists=('submission_exists', 'sum'),
    any_medal=('any_medal', 'sum'),
    gold_medal=('gold_medal', 'sum'),
    silver_medal=('silver_medal', 'sum'),
    bronze_medal=('bronze_medal', 'sum'),
    above_median=('above_median', 'sum')
).reset_index()
# grouped_df_bo

In [19]:
bo_full_df

,competition_id,score,gold_threshold,silver_threshold,bronze_threshold,median_threshold,any_medal,gold_medal,silver_medal,bronze_medal,above_median,submission_exists,valid_submission
0,aptos2019-blindness-detection,0.88578,0.930508,0.919654,0.914492,0.888912,False,False,False,False,False,True,True
1,dog-breed-identification,NaN,0.000500,0.005390,0.045980,0.472050,False,False,False,False,False,True,False
2,detecting-insults-in-social-commentary,NaN,0.833210,0.823070,0.791110,0.778420,False,False,False,False,False,True,False
3,aerial-cactus-identification,0.99998,1.000000,1.000000,1.000000,0.999100,False,False,False,False,True,True,True
4,denoising-dirty-documents,NaN,0.017940,0.026090,0.045170,0.073250,False,False,False,False,False,True,False
5,leaf-classification,0.19225,0.000000,0.007910,0.015260,0.108345,False,False,False,False,False,True,True
6,histopathologic-cancer-detection,0.98633,0.983500,0.979800,0.973800,0.947700,True,True,False,False,True,True,True
7,dogs-vs-cats-redux-kernels-edition,3.82126,0.038820,0.050380,0.061270,0.122160,False,False,False,False,False,True,True
8,jigsaw-toxic-comment-classification-challenge,0.97187,0.987400,0.986680,0.986390,0.980790,False,False,False,False,False,True,True
9,mlsp-2013-birds,0.67635,0.935270,0.900380,0.873720,0.865720,False,False,False,False,False,True,True


In [20]:


# Convert counts to percentages
grouped_df['valid_submissions'] = (grouped_df['valid_submissions'] / grouped_df['total_competitions']) * 100
grouped_df['submission_exists'] = (grouped_df['submission_exists'] / grouped_df['total_competitions']) * 100
grouped_df['any_medal'] = (grouped_df['any_medal'] / grouped_df['total_competitions']) * 100
grouped_df['gold_medal'] = (grouped_df['gold_medal'] / grouped_df['total_competitions']) * 100
grouped_df['silver_medal'] = (grouped_df['silver_medal'] / grouped_df['total_competitions']) * 100
grouped_df['bronze_medal'] = (grouped_df['bronze_medal'] / grouped_df['total_competitions']) * 100
grouped_df['above_median'] = (grouped_df['above_median'] / grouped_df['total_competitions']) * 100
# grouped_df.to_csv("/home/xuyang1/mle-bench-aide-gpt-4o-2024-mini/res_gpt_4o_aide_mini_each2.csv", index=False)
# grouped_df


grouped_df_bo['valid_submissions'] = (grouped_df_bo['valid_submissions'] / grouped_df_bo['total_competitions']) * 100
grouped_df_bo['submission_exists'] = (grouped_df_bo['submission_exists'] / grouped_df_bo['total_competitions']) * 100
grouped_df_bo['any_medal'] = (grouped_df_bo['any_medal'] / grouped_df_bo['total_competitions']) * 100
grouped_df_bo['gold_medal'] = (grouped_df_bo['gold_medal'] / grouped_df_bo['total_competitions']) * 100
grouped_df_bo['silver_medal'] = (grouped_df_bo['silver_medal'] / grouped_df_bo['total_competitions']) * 100
grouped_df_bo['bronze_medal'] = (grouped_df_bo['bronze_medal'] / grouped_df_bo['total_competitions']) * 100
grouped_df_bo['above_median'] = (grouped_df_bo['above_median'] / grouped_df_bo['total_competitions']) * 100
# grouped_df_bo.to_csv("/home/xuyang1/mle-bench-aide-gpt-4o-2024-mini/res_bo_full_each2.csv", index=False)
# grouped_df_bo


In [21]:
# Made Submission Rate

comp_made_rate = grouped_df['submission_exists'].sum() / len(grouped_df)
comp_made_rate_bo = grouped_df_bo['submission_exists'].sum() / len(grouped_df_bo)
print("comp_made_rate:", comp_made_rate)
print("comp_made_rate_bo:", comp_made_rate_bo,'\n')

# Valid Submission Rate
comp_finished_rate = grouped_df['valid_submissions'].sum() / len(grouped_df)
comp_finished_rate_bo = grouped_df_bo['valid_submissions'].sum() / len(grouped_df_bo)
print("comp_finished_rate:", comp_finished_rate)
print("comp_finished_rate_bo:", comp_finished_rate_bo,'\n')

# Above Median Rate
above_median_rate = grouped_df['above_median'].sum() / len(grouped_df)
above_median_rate_bo = grouped_df_bo['above_median'].sum() / len(grouped_df_bo)
print("above_median_rate:", above_median_rate)
print("above_median_rate_bo:", above_median_rate_bo,'\n')

# Gold Medal Rate
gold_medal_rate = grouped_df['gold_medal'].sum() / len(grouped_df)
gold_medal_rate_bo = grouped_df_bo['gold_medal'].sum() / len(grouped_df_bo)
print("gold_medal_rate:", gold_medal_rate)
print("gold_medal_rate_bo:", gold_medal_rate_bo,'\n')

# Silver Medal Rate
silver_medal_rate = grouped_df['silver_medal'].sum() / len(grouped_df)
silver_medal_rate_bo = grouped_df_bo['silver_medal'].sum() / len(grouped_df_bo)
print("silver_medal_rate:", silver_medal_rate)
print("silver_medal_rate_bo:", silver_medal_rate_bo,'\n')

# Bronze Medal Rate
bronze_medal_rate = grouped_df['bronze_medal'].sum() / len(grouped_df)
bronze_medal_rate_bo = grouped_df_bo['bronze_medal'].sum() / len(grouped_df_bo)
print("bronze_medal_rate:", bronze_medal_rate)
print("bronze_medal_rate_bo:", bronze_medal_rate_bo,'\n')

# Any Medal Rate
any_medal_rate = grouped_df['any_medal'].sum() / len(grouped_df)
any_medal_rate_bo = grouped_df_bo['any_medal'].sum() / len(grouped_df_bo)
print("any_medal_rate:", any_medal_rate)
print("any_medal_rate_bo:", any_medal_rate_bo,'\n')


comp_made_rate: 59.09090909090909
comp_made_rate_bo: 86.36363636363636 

comp_finished_rate: 50.0
comp_finished_rate_bo: 68.18181818181819 

above_median_rate: 18.181818181818183
above_median_rate_bo: 18.181818181818183 

gold_medal_rate: 4.545454545454546
gold_medal_rate_bo: 9.090909090909092 

silver_medal_rate: 4.545454545454546
silver_medal_rate_bo: 4.545454545454546 

bronze_medal_rate: 0.0
bronze_medal_rate_bo: 0.0 

any_medal_rate: 9.090909090909092
any_medal_rate_bo: 13.636363636363637 



In [ ]:
markdown_table = f"""
| Model      | Made Submission (%) | Valid Submission (%) | Above Median (%) | Bronze (%) | Silver (%) | Gold (%) | Any Medal (%) |
|------------|---------------------|-----------------------|------------------|------------|------------|----------|---------------|
| AIDE (Ours, gpt-4o_2024-08-06) | {comp_made_rate:.1f}             | {comp_finished_rate:.1f}               | {above_median_rate:.1f}           | {bronze_medal_rate:.1f}      | {silver_medal_rate:.1f}      | {gold_medal_rate:.1f}    | {any_medal_rate:.1f}         |
# | AIDE-BO (Ours, gpt-4o_2024-08-06) | {comp_made_rate_bo:.1f}             | {comp_finished_rate_bo:.1f}               | {above_median_rate_bo:.1f}           | {bronze_medal_rate_bo:.1f}      | {silver_medal_rate_bo:.1f}      | {gold_medal_rate_bo:.1f}    | {any_medal_rate_bo:.1f}         |
| AIDE-BO-gpu (Ours, gpt-4o_2024-08-06) | {comp_made_rate_bo:.1f}             | {comp_finished_rate_bo:.1f}               | {above_median_rate_bo:.1f}           | {bronze_medal_rate_bo:.1f}      | {silver_medal_rate_bo:.1f}      | {gold_medal_rate_bo:.1f}    | {any_medal_rate_bo:.1f}         |

"""

print(markdown_table)

| Model      | Made Submission (%) | Valid Submission (%) | Above Median (%) | Bronze (%) | Silver (%) | Gold (%) | Any Medal (%) |
|------------|---------------------|-----------------------|------------------|------------|------------|----------|---------------|
| AIDE (OpenAI, gpt-4o_2024-08-06) | 82.8             | 66.7               | 30.4           | 3.4      | 6.0      | 9.6    | 19.0         |
| AIDE-w.o GPU (Ours, gpt-4o_2024-08-06) | 59.1             | 50.0               | 18.2           | 0.0      | 4.5      | 4.5    | 9.1         |
| AIDE-BO-w.o GPU (Ours, gpt-4o_2024-08-06) | 72.7             | 66.7               | 18.2           | 0.0      | 4.5      | 4.5    | 9.1         |
| AIDE-BO-w GPU (Ours, gpt-4o_2024-08-06) | 86.4             | 68.2               | 18.2           | 0.0      | 4.5      | 9.1    | 13.6         |

| Model      | Made Submission (%) | Valid Submission (%) | Above Median (%) | Bronze (%) | Silver (%) | Gold (%) | Any Medal (%) |
|------------|---------------------|-----------------------|------------------|------------|------------|----------|---------------|
| AIDE (OpenAI, gpt-4o_2024-08-06) | 82.8             | 66.7               | 30.4           | 3.4      | 6.0      | 9.6    | 19.0         |
| AIDE (Ours, gpt-4o_2024-08-06) | 59.1             | 50.0               | 18.2           | 0.0      | 4.5      | 4.5    | 9.1         |
| AIDE-BO (Ours, gpt-4o_2024-08-06) | 72.7             | 66.7               | 18.2           | 0.0      | 4.5      | 4.5    | 9.1         |

In [14]:
# Create a table comparing scores for competitions with valid submissions
valid_comps = grouped_df[grouped_df['valid_submissions'] > 0]['competition_id'].unique()

if_BO_help_improve = []
for comp in valid_comps:
    if competition_score_dict[comp][0] > competition_score_dict_bo[comp][0]:
        if competition_score_dict_bo[comp][1]:
            if_BO_help_improve.append(True)
        else:
            if_BO_help_improve.append(False)
    else:
        if competition_score_dict_bo[comp][1]:
            if_BO_help_improve.append(False)
        else:
            if_BO_help_improve.append(True)

score_comparison = pd.DataFrame({
    'Competition': valid_comps,
    # 'AIDE (OpenAI)': [0.0] * len(valid_comps),  # Placeholder since we don't have OpenAI scores
    'AIDE (Ours)': [competition_score_dict[comp][0] for comp in valid_comps],
    'AIDE-BO': [competition_score_dict_bo[comp][0] for comp in valid_comps],
    'is_lower_better': [competition_score_dict_bo[comp][1] for comp in valid_comps],
    'BO help improve?': if_BO_help_improve
})

# Format scores to 4 decimal places
score_comparison['AIDE (Ours)'] = score_comparison['AIDE (Ours)'].map('{:.4f}'.format)
score_comparison['AIDE-BO'] = score_comparison['AIDE-BO'].map('{:.4f}'.format)

print("\nScore Comparison for Valid Submissions:")
print(score_comparison.to_string(index=False))


TypeError: '>' not supported between instances of 'float' and 'NoneType'

In [15]:
if_BO_help_improve

[False, False, False, False]

In [16]:
grouped_df

,competition_id,total_competitions,valid_submissions,submission_exists,any_medal,gold_medal,silver_medal,bronze_medal,above_median
0,aerial-cactus-identification,1,100.0,100.0,0.0,0.0,0.0,0.0,100.0
1,aptos2019-blindness-detection,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,denoising-dirty-documents,1,0.0,100.0,0.0,0.0,0.0,0.0,0.0
3,detecting-insults-in-social-commentary,1,0.0,100.0,0.0,0.0,0.0,0.0,0.0
4,dog-breed-identification,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,dogs-vs-cats-redux-kernels-edition,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,histopathologic-cancer-detection,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,jigsaw-toxic-comment-classification-challenge,1,100.0,100.0,0.0,0.0,0.0,0.0,0.0
8,leaf-classification,1,100.0,100.0,0.0,0.0,0.0,0.0,0.0
9,mlsp-2013-birds,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
